# Notebook for Adding NLP Data to Database

In [41]:
import pandas as pd
import numpy as np

## Connect to Database

In [42]:
import mysql.connector

In [43]:
endpoint = "seratestdatabase.c4cjk1vto1om.us-east-2.rds.amazonaws.com"
port = "3306"
usr = "admin"
pswd = "your_password_here"
region = "us-east-2b"
dbname = "teachsim"

In [44]:
cnx = mysql.connector.connect(user=usr, password=pswd, host=endpoint, database=dbname)

In [45]:
cursor = cnx.cursor(buffered=True)

### Test Connection

In [46]:
cursor.execute("SELECT * FROM Fidelity_Measures;")
result = cursor.fetchall()
colnames = [x[0] for x in cursor.description]
pd.DataFrame(result, columns=colnames)

,doctype,filename,study,model,skill,coach,text,cleaned_vectorized_document,similarity_score
0,transcript,10-2C,Behavior Study 1,behavior,2.0,None,"Okay, so I think that wins. Dont like the simu...","[0.5194993709632443, 0.023576768930381554, 0.4...",0.2188967631854828
1,transcript,103-2C,Behavior Study 1,behavior,2.0,None,alright. Why dont you have a sit and well-- we...,"[0.4031709161284383, -0.08087668041599247, 0.2...",0.1458457998040087
2,transcript,2019_101_5C_Transcript,Behavior Study 2,behavior,2.0,person5,"Hello. Okay, so how do you think at first run ...","[0.5741149744532488, -0.17514627745491707, -0....",0.1962542624321331
3,transcript,2019_106_5C_Transcript,Behavior Study 2,behavior,2.0,person3,So tell me how you feel about your first simul...,"[0.5623479361279925, -0.18734489200306162, -0....",0.2201094467777059
4,transcript,2019_119_5C_Transcript,Behavior Study 2,behavior,2.0,None,"alright. So, how are you feeling about Okay. W...","[0.5764204590246595, -0.03895625224437529, -0....",0.2520476945940342
5,transcript,2019_20_5C_Transcript,Behavior Study 2,behavior,2.0,person1,"alright, so come on over. How do you feel that...","[0.6059433028211592, -0.1602070625338875, 0.01...",0.2514485797592812
6,transcript,2019_22_5C_Transcript,Behavior Study 2,behavior,2.0,person5,Just go ahead and take a seat. Youre fine. You...,"[0.604000330114129, -0.06524464897980672, -0.1...",0.2643317150237819
7,transcript,2019_34_5C_Transcript,Behavior Study 2,behavior,1.0,person7,"So, how are you feeling about that first ? Tha...","[0.6361390756591526, -0.04385806229881961, -0....",0.300864457526095
8,transcript,2019_38_5C_Transcript,Behavior Study 2,behavior,2.0,person7,How are you feeling about the first simulation...,"[0.5822116460450054, -0.01135436117190061, -0....",0.2868702584122339
9,transcript,2019_42_5C_Transcript,Behavior Study 2,behavior,2.0,person3,"Aside from terrible, before I tell you my feed...","[0.5782576176773634, -0.1845167081384388, -0.1...",0.2432368806421148


In [31]:
cursor.execute("DELETE FROM Fidelity_Measures;")

## Functions

In [2]:
def getColumnNames(tableName, cursor):
    query = "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = N'" + tableName + "'"
    cursor.execute(query)
    result = cursor.fetchall()
    result2 = [x[0] for x in result]
    
    return result2

In [3]:
def overlapColumns(cols1, cols2):
    overlap = []
    for col in cols1:
        if col in cols2:
            overlap = overlap + [col]
    return overlap

In [4]:
def getExistingIds(tableName, cursor):
    query = "SELECT filename FROM " + str(tableName)
    cursor.execute(query)
    result = cursor.fetchall()
    existing_ids = [x[0] for x in result]
    return existing_ids

In [5]:
def fixStrings(df):
    types = dict(df.dtypes)
    cols = [x for x in df.columns if types[x] == np.object]
    for index, row in df.iterrows():
        for col in cols:
            value = row[col]
            if type(value) == str:
                new = value.replace("'", "")
                new2 = new.replace('"', '')
                df.loc[index, col] = new2
    return df

In [6]:
def list_queries(df, columns, tableName, existing_ids):
    queries = []
    for index, row in df.iterrows():
        pid = row['filename']
        valid_pid = False
        if type(pid) == str:
            valid_pid = True
                
        # if the row for that participant already exists, just update it
        if pid in existing_ids:
            req_update = False
            query = "UPDATE " + tableName + " SET "
            for col in columns:
                newval = row[col]
                if type(newval) == str:
                    query = query + col + " = '" + newval + "' , "
                    if col != "filename":
                        req_update = True
                elif np.isnan(float(newval)) == False: 
                    query = query + col + " = " + str(newval) + " , "
                    if col != "id_participant":
                        req_update = True
            if req_update == True:
                query = query[:-2]
                query = query + "WHERE filename = '" + row["filename"] + "';"
                queries = queries + [query]
        
        # if that participant is not in the database, create/replace the row
        elif valid_pid == True:
            req_update = False
            query = "REPLACE INTO " +  tableName + " "
            cols = "("
            vals = "("
            for col in columns:
                val = row[col] 
                if type(val) == str:
                    cols = cols + str(col) + ", "
                    vals = vals + "'" + val + "', "
                    if col != "filename":
                        req_update = True
                elif np.isnan(float(val)) == False: 
                    cols = cols + str(col) + ", "
                    vals = vals + str(val) + ", "
                    if col != "filename":
                        req_update = True
            query = query + cols[:-2] + ") " + 'VALUES ' + vals[:-2] + ");"
            if req_update == True:
                queries = queries + [query]
                existing_ids = existing_ids + [row['filename']]
            
    return queries


In [7]:
def generateQueries(df, tableName, cursor):
    df_cols = df.columns
    tab_cols = getColumnNames(tableName, cursor)
    overlap = overlapColumns(df_cols, tab_cols)
    existing_ids = getExistingIds(tableName, cursor)
    cleaned = fixStrings(df)
    query_list = list_queries(cleaned, overlap, tableName, existing_ids)
    return query_list

## Change Working Directory

In [8]:
import os
os.getcwd()

'/Users/jmachita03/Documents/GitHub/MSDS_SERA_capstone/RDS'

In [9]:
os.chdir("/Users/jmachita03/Desktop/Capstone/NLP")

In [15]:
data = pd.read_csv("normal_comparison_output_status.csv")

In [16]:
data.head()

,doctype,index,filename,study,model,skill,coach,text,cleaned_vectorized_document,similarity_score
0,transcript,21,2019_5_5C_Transcript,Behavior Study 2,behavior,1,person1,Yeah that can be tough. I totally agree. Usual...,"[0.6198429304314097, -0.16436602338468467, -0....",0.270622
1,transcript,18,2019_34_5C_Transcript,Behavior Study 2,behavior,1,person7,"So, how are you feeling about that first ? Tha...","[0.6361390756591526, -0.04385806229881961, -0....",0.300864
2,transcript,1,103-2C,Behavior Study 1,behavior,2,NaN,alright. Why don't you have a sit and we'll-- ...,"[0.4031709161284383, -0.08087668041599247, 0.2...",0.145846
3,transcript,2,6-2C,Behavior Study 1,behavior,2,NaN,So how do you feel about that? Really hard. Wh...,"[0.5472509215170646, -0.07767101051949, 0.2492...",0.244462
4,transcript,9,2019_58_5C_Transcript,Behavior Study 2,behavior,2,person3,alright. Nice job. Like so many questions for ...,"[0.5707874299747961, -0.07655687445423678, -0....",0.277304


In [18]:
queries = generateQueries(data, "Fidelity_Measures", cursor)

In [19]:
for query in queries:
    cursor.execute(query)

In [20]:
cursor.execute("SELECT * FROM Fidelity_Measures;")
result = cursor.fetchall()
colnames = [x[0] for x in cursor.description]
pd.DataFrame(result, columns=colnames)

,doctype,filename,study,model,skill,coach,text,cleaned_vectorized_document,similarity_score
0,transcript,10-2C,Behavior Study 1,behavior,2.0,None,"Okay, so I think that wins. Dont like the simu...","[0.5194993709632443, 0.023576768930381554, 0.4...",0.2188967631854828
1,transcript,103-2C,Behavior Study 1,behavior,2.0,None,alright. Why dont you have a sit and well-- we...,"[0.4031709161284383, -0.08087668041599247, 0.2...",0.14584579980400875
2,transcript,2019_101_5C_Transcript,Behavior Study 2,behavior,2.0,person5,"Hello. Okay, so how do you think at first run ...","[0.5741149744532488, -0.17514627745491707, -0....",0.19625426243213315
3,transcript,2019_106_5C_Transcript,Behavior Study 2,behavior,2.0,person3,So tell me how you feel about your first simul...,"[0.5623479361279925, -0.18734489200306162, -0....",0.220109446777706
4,transcript,2019_119_5C_Transcript,Behavior Study 2,behavior,2.0,None,"alright. So, how are you feeling about Okay. W...","[0.5764204590246595, -0.03895625224437529, -0....",0.2520476945940342
5,transcript,2019_20_5C_Transcript,Behavior Study 2,behavior,2.0,person1,"alright, so come on over. How do you feel that...","[0.6059433028211592, -0.1602070625338875, 0.01...",0.25144857975928125
6,transcript,2019_22_5C_Transcript,Behavior Study 2,behavior,2.0,person5,Just go ahead and take a seat. Youre fine. You...,"[0.604000330114129, -0.06524464897980672, -0.1...",0.2643317150237819
7,transcript,2019_34_5C_Transcript,Behavior Study 2,behavior,1.0,person7,"So, how are you feeling about that first ? Tha...","[0.6361390756591526, -0.04385806229881961, -0....",0.30086445752609503
8,transcript,2019_38_5C_Transcript,Behavior Study 2,behavior,2.0,person7,How are you feeling about the first simulation...,"[0.5822116460450054, -0.01135436117190061, -0....",0.2868702584122339
9,transcript,2019_42_5C_Transcript,Behavior Study 2,behavior,2.0,person3,"Aside from terrible, before I tell you my feed...","[0.5782576176773634, -0.1845167081384388, -0.1...",0.2432368806421149


## Commit and Close

In [47]:
cnx.commit()
cnx.close()